# Changes in thermal expansion coefficient

## Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import seaborn as sns
import xarray as xr
import tqdm
import pathlib
import cmocean
import os
import copy
import time
import gsw

# Import custom modules
import src.utils

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## bump up DPI
mpl.rcParams["figure.dpi"] = 100

## get filepaths
DATA_FP = pathlib.Path(os.environ["DATA_FP"])
SAVE_FP = pathlib.Path(os.environ["SAVE_FP"])

## funcs

In [ ]:
def avg_upper_ocn(T, H=70, lon_range=[140, 280]):
    """average upper ocean above threshold"""
    T_ = T.sel(z_t=slice(None, H), longitude=slice(*lon_range))

    return T_.mean(["z_t", "longitude"])

## Load data

In [ ]:
## load data
forced, _ = src.utils.load_consolidated()

## subset for temperature
T = forced[["T", "T_comp"]]

## get windowed T
T = src.utils.get_windowed(T, stride=120)

## Compute

### Avg. temperature over near-surface layer

In [ ]:
T_upper = src.utils.reconstruct_clim(T, fn=avg_upper_ocn)["T"]
T_upper_mean = T_upper.mean("month")

### Compute thermal expansion

In [ ]:
## specify reference values
SA = 35  # g/kg
P = 35  # dbar
H = 70  # m

## reference values for salinity, pressure
nyear = len(T_upper.year)
sa = SA * np.ones(nyear)
p = P * np.ones(nyear)

## compute alpha
alpha = xr.ones_like(T_upper_mean)
alpha.values = gsw.alpha(SA=sa, CT=T_upper_mean.values, p=p)

## get fractional change
delta_a = src.utils.frac_change(alpha, inv=False)

### Plot

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(T_upper.year, delta_a)
ax.set_ylabel("Frac. change")
plt.show()